# Advanced Analytics(AA) COE OCR and Data Extraction Process Overview


### Stage 1: Preprocess ,  OCR, and Transform Original Documents
    #### 1. Identify if PDF is searchable or unsearchable 
    #### 2. Repair , Clean and enhance original pdf(with image) 
    #### 3. Use LSTMs + OSD configuration from Tesseract to OCR PDF and Create searchable document 
    #### 4. Convert and Save searchable PDF into Formatted Text file preserving all spacing and formatting of original pdf 
    #### 5. Load Formated txt file and store each row individually into a list 

### Stage 2: Attribute Extraction 
    #### 1. Create Dictonary of Reg-ex patterns to identify attribute and table locations
    #### 2. Search rows in doc for a match to a reg-ex pattern associated with attribute in question (row, startpos, endpos)
    #### 3. Parse attribute value from text based on loc relationship with attribute header and apply Checks to filter out FPs
            # 1. Check text to right of header (first word > 5 characters, first 2 words are not text, text contains num) 
            # 2. if no text found to right , check text BELOW attribute header using header loc as starting pos for next line 
    #### 4. Apply check to ensure correct attribute extracted (check:  1 group of text with  numbers, chars,  special chars
    
### Stage 3: Table Extraction 
    #### 1. Find starting row of table using regex patterns stored in Dictonary that indicate table start 
    #### 2. Find ending location of table using patterns associateed with end of table for a particular company 
    #### 3. Extract information between start and end row of table using delimiter(double space, works 80% of time)
    #### 3. For ~100% Accuracy, simply define character ranges that are associated with the table for each comapany
        
##### Date Creation: 9/21
##### Last Edited: 10/5
##### Authors: Zack Carideo + Ran Zhou    

In [2]:
#Import Modules
import os 
import re 
from collections import OrderedDict
from operator import itemgetter

#from StringIO import StringIO
import stat
import sys, getopt
from subprocess import Popen, PIPE
import subprocess
#import pdftotext
from re import finditer
import fileinput

## 1.Identify Unserachable PDFS to Be Converted to Searchable 

In [3]:
#Path tied to converted ocrmypdf pdf files 
pdf_path = "//home//docadmin//python//ocr//samples//SearchablePDF_Sandwiches//"

#Destination path for formatted text output
txt_path = "//home//docadmin//python//ocr//samples//Converted_txtfiles//"

#pdf file names found in the pdf_path location
files = getfiles(pdf_path, ".pdf")

#empty list to hold paths to txt files 
pdftextpaths=[]

#loop through each OCRMYPDF converted pdfs, and convert to text with layout of original pdf maintained as text 
for i in range(len(files)):
    txtpath = txt_path + files[i].replace("_V2searchable.pdf",".txt")
    pdfpath = pdf_path + files[i]
    
    #append all text paths
    pdftextpaths.append(txtpath)
    
    #convert all pdfs to text and preserve layout 
    #os.system('pdftotext -layout '+pdfpath+' '+txtpath)
    
#loop through all searchable pdfs and convert to text with layout of original pdf maintained as text 
for i in range(len(SearchablePDFPaths)):
    txtpath = txt_path + str(SearchablePDFPaths[i].rsplit("//", 1)[1]).replace(".pdf","")+".txt"
    #os.system('pdftotext -layout '+SearchablePDFPaths[i]+' '+txtpath)
    pdftextpaths.append(txtpath)

FileNotFoundError: [WinError 53] The network path was not found: '//home//docadmin//python//ocr//samples//SearchablePDF_Sandwiches//'

# #################
# BEGIN GDS GUIDE#
# #################

### 4.Read in Text File to be Processed and split it into individual lines

In [4]:
############
###INPUTS###
############
#specify location of all text files converted using AA COE OCR Function 
#txt_path = "//home//docadmin//python//ocr//samples//Converted_txtfiles//"
txt_path="C://Users//caridza//Desktop//pythonScripts//OCR//ServerBackUpCodeRepo//txt_output//"
#specify the file number you want to process(variable or needs to be an iterable object)
filenum =13

#specify how file is delimited (defaults to "\n")
delim = "\n"

#encoding used (if special characters are present you must use utf_8_sig)
encoding = "utf_8_sig"

############
###INPUTS###
####END#####

#Generates list of all txt files in the directory where converted text files are stored 
files = getfiles(txt_path, ".txt")

#Open file 
f =open(txt_path+files[filenum],encoding = encoding)

#Read file as string
a= f.read()

#Split lines into list based on new line indicator
b = a.split(delim)

#remove all completely white lines (helps postprocessing and identifying values that are below associated attribute)
c = [x for x in b if x]
c[:18]

['                                             FERMI RESEARCH ALLIANCE LLC.                                                               IMPORTANT                          PURCHASE ORDER',
 '                                      FERMI NATIONAL ACCELERATOR LABORATORY                                                 PLEASE PUT THE ORDER NUMBER ON',
 '                                                                                                                            ALL INVOICES, SHIPPING NOTICES,                NO. 636617',
 '#8',
 '                                              P.0.BOX 500 BATAVIA, ILLINOIS 60510',
 '                                                                                                                            CORRESPONDENCE AND THE EXTERIOR',
 '                                                       PHONE : 630 840—3000                                                 OF ALL SHIPPING CONTAINERS:',
 '                                                      

In [ ]:
#NOTE: NEED TO FILTER OUT BAD QUALITY PO's UPFRONT TO STAGE FOR MANUAL PROCESSING
#      Option 1: FILTER PO's by Company Name so that we only process PO's from companies we know we can get right 

#SHIPPING TO RULES 
#Grey Matters: 3 lines below ship to (look for multiple spaces on the extremes of the strings below header to define character space to consider on each line below header)
#Kimberly-Clark: 3 lines below "shipping Adderss" (first line with whitespace indicates end of address)
#Sabritas: 5 lines below "SEND TO:"(first line below is blank + 4 lines below are address)(STOP at line before line containing "REQUISITIONER DEPARTMENT")
#Gexpro: 6 liens below "SHIPO TO:"(first 2 lines below are blank + 4 lines below are address)(first line of whitespace indicates end of address)
#CBPacific: 5 lines below "Ship To":(format of lines: rand?,comapny,address, city state zipcode, Country)
#Cooper Tire & Rubber Co: 5 lines right and below Ship To:(1 line to right, and 4 below that with left margin all aligned and ends line above "Buyer's Name")
#LAFARGE: 5 lines below "Ship/Service To:"(stop at white space or line before table begins)
#FERMILAB: SHIP TO is spread out on multiple lines next to adderss, so find SHIP, then find the start char of the next word to right, look up 3 and down 3 rows from that and find the starting positions of thos strings (find all lines with non "" starts)

# BEGIN Attribute and Table Extraction 

## 5.Create Dictonary to hold all patterns used to identify attribute names 

### GDS Note: The below template provides a guideline for creating a dictonary of key value pairs to find "PO Number" ,  identify the starting poisition of a table, and generate control checkpoints to ensure the string found matches the format of the attribute. This dictonary must be expanded to include a key value pair for all attributes in scope. 

#### 1. Each attribute that must be found requires a key:value pair in the below dictonary 
##### (Key,Value): Key = Name of attribute we are looking for , Value = Regex patterns used to identify the attribute 
#### 2. Each control / validation check for each attribute requires a key value pair in the below dictonary 
##### (Key,Value): Key = Name of control we are applying , Value = Regex patterns used to validate the string returned has correct format/length 
#### 3.Each COMPANY will require unique END of table key:value pair in the dicontary. Currently we have illistrated how to use identify the start and end of table for GREYMATTER, similar approach must be taken for each company 
#### 4. As multiple attributes are added GDS may need multiple dictonaries(or dictonary of dictonaries), 1 to hold key/value attribute search rules, 1 to hold rules to validate results 

In [5]:
#create empty dictonary to hold key value pairs(attribute : regex)
#Note:holds all attributes and the regex used to find each
LookupTable = {}

#dictonary key value pairs (Attribute:Regex)

#ATTRIBUTE: PO NUMBER 
#represents all rules required to identify the attribute PO NUMBER across all variations of POs 
LookupTable['PO NUMBER']=re.compile(r'Order Number:|CHANGE PURCHASE ORDER:|\s{4}(P/O NUMBER)|P.O. NO.:|P.O. #:|PO #|PO Number|PO Number:| Purchase Order No:(#)?|PURCHASE ORDER(#)?')

LookupTable['SHIPTO']=re.compile(r'\s{4,}Ship To|Ship To\s{4,}|Ship To:|\s{4}SHIP(?! VIA)|Shipping Address|Ship-to-Address|\s{4}SHIP TO:|Shipping Adderss\s{4}|Ship/Service To:\s{4}|\s{4}SEND TO:\s{4}')

LookupTable['COMPANY']=re.compile(r'AL HOSN|CB Pacific|BRIDGESTONE|Volvo|Cooper Tire|Gray Matter|FERMI|Gexpro|INTERTAPE POLYMER CORP|KC Global|Lafarge|DESCO|Sabritas')

#ATTRIBUTE: TABLE(identify START line of table) 
LookupTable['TABLE']=re.compile(r'Item #+( +PARTICULAR)(.*)|\s+ITEM+( +P. REQ.)(.*)|\s+ITEM+( +QTY)+( +UM)(.*)|^Item+( +Material/Service)+( Description)(.*)|\s+Line+( +Quantity)+( +Item)(.*)|\s+LN+( +ORDER)+( +QTY)(.*)|^Item+( +Quantity)(.*)|^Line+( +Item)(.*)|^Item Nr+( +Description)(.*)')

#CONTROL: Rule to ensure  PO number returned is not 2 full non integer words
LookupTable['MultipleWordTest']=re.compile(r'^([a-zA-Z]+\s){2}')

#ATTRIBUTE: TABLE(identify END line of a table based on pattern in company PO that can be used to identify EOT) 
#Note: This is comapny specific, i have shown for Grey Matters Inc below. 
#Note: in all GREYMATTER PO's the table ends at the line before the string "TOTAL" or "SUBTOTAL" is found depending on the layout of the PO (2 distinct layouts identifeid for GreyMatter)
LookupTable['GREYTABLEEND1']=re.compile(r'(?<!\S)Subtotal(?!\S)')  #Subtotal logic
LookupTable['GREYTABLEEND2']=re.compile(r'(^|\s{4,})Total(\s+|$)') #Total Logic (at least 4 spaces before the word being searched for)

### Select attribute to process 
#### Note: script is setup to process one element at a time(no loop), once all rules have been generated for 
#### the 16 attributes we will need to wrap the below search , find, and aggreation functions into a loop

In [6]:
#############
####INPUT####
#############

#Attribute = 'PO NUMBER'
Attribute = 'SHIPTO'
#############
####INPUT####
#####END#####

In [7]:
# Visually look at structure and output used to loop through each line in document and pinpoint the locaiton of the attribute 
# header that we are interested in. The below code loops through each line of document and identify where matches for PO number have been found       
for i in range(len(c)):
    print(str(i),re.search(LookupTable[Attribute],c[i]))

0 None
1 None
2 None
3 None
4 None
5 None
6 None
7 None
8 None
9 None
10 <_sre.SRE_Match object; span=(112, 120), match='    SHIP'>
11 None
12 None
13 None
14 None
15 None
16 None
17 None
18 None
19 None
20 None
21 None
22 None
23 None
24 None
25 None
26 None
27 None
28 None
29 None
30 None
31 None
32 None
33 None
34 None
35 None
36 None
37 None
38 None
39 None
40 None
41 None
42 None
43 None
44 None
45 None
46 None
47 None
48 None
49 None
50 None
51 None
52 None
53 None
54 None
55 None
56 None
57 None
58 None
59 None
60 None
61 <_sre.SRE_Match object; span=(117, 125), match='    SHIP'>
62 None
63 None
64 None
65 None
66 None
67 None
68 None
69 None
70 None
71 None
72 None
73 None
74 None
75 None
76 None
77 None
78 None
79 None
80 None
81 None
82 None
83 None
84 None
85 None
86 None
87 None
88 None
89 None
90 None
91 None
92 None
93 None
94 None
95 None
96 None


## 6.Store match information for the attribute in question into a new dictonary

### NOTE: identifies alll matches found for attribute in question and returns the line #, charter start/end position and the value the reg-ex matched on into a list. This informaiton is used to identify the starting location within the document that we will use to find the 'value' of the attribute header we are searching for.  

In [8]:
#CREATE EMPTY DICTONARY TO HOLD KEY VALUE PAIRS RETURNED FOR EACH ATTRIBUTE(contains all matches for the element in question)
#Key = ATtribute Name , Value = Locational info of match
test ={}

#CREATE EMPTY LIST TO HOLD INTERMEDIATE MATCH RESULTS(compiled within the FOR loop) 
#the values in this list are passed to the dictonary "test" after the loop has covered all rows in document 
list1 = []

#for every line in the PO look for pattern in text representative of the reg-ex patterns defined in the LookUpTable Dictonary
#and store each match found to a list. Once all matches have been stored to the list, assign each match as a key:value pair in the dicontary test
for i in range(len(c)):
   
    # apply regex pattern associated with attribute in question 
    # and for each pattern match found in line i generate locational information 
    # to use in parsing / finding the associated attribute value 
    for match in finditer(LookupTable[Attribute], c[i]):
     
        #output nested list with tuple of locational information about the match 
        #Note:creates a 3 element tuple with the information on locaiton of string
        list1.append(([str(i)],[match.span()], [match.group()]))

#ASSIGN DICTONARY VALUES FOR THE KEY BEING PROCESSED(here KEY = PO NUMBR) 
#Note: we can change it to test[list(LookupTable.keys())[1]] = list1 #change 1 to an iterable object to make dynamic
#      but because order is not gaurenteed in dictonaries we may want to leave it as shown below 

#if no matches were found raise a flag as the value of the tuple
if not list1: 
    NoMatchFound = 1 
else: 
    NoMatchFound = 0
    test[Attribute] = list1 

    #attribute results associated with 
    list(test.keys())[0]

    #all matches and associated info teid to attribute 
    for i in test.keys():
        for j in test[i]:
            print(j)

(['10'], [(112, 120)], ['    SHIP'])
(['61'], [(117, 125)], ['    SHIP'])


In [254]:
#dictonary syntax 
test.items()
test.keys()
test.values()

str(test)
len(test)
type(test)

' ORDER DATE              BUYER                                     TERMS: FROM RECEIPT OFGOODS                                F.0.8                           SHIP VIA:                        REQUESTOR'

In [15]:
########################
###PARSING FOR SHIP TO## 
########################
import operator
if NoMatchFound ==0: 
    matchdetails = []
    #look through each match stored in "test" dictonary(from prior step) and determine 
    #if the expected attribute value is to the right or below the attribute header/name
    for i in test.keys():
        for j in test[i]:

            #1.Aggregated details on matched strings  
            matchdetails.append(
                                   ( int(j[0][0])                             #Line in text inwhich the match was found 
                                   , wordCount(stripwhite(c[int(j[0][0])]))   #Totalwords in line string was found
                                   , len(c[int(j[0][0])])                     #Length of characters in the line the match was found 
                                   , j[1][0]                                  #Start and end position of the matched pattern found 
                                   , j[2][0]                                  #Actual value the pattern matched on 
                                   ))

            #2.processing each match to find true result if the attribute being processed "ATTRIBUTE" is the attribute match results have been populated for(This should always be true)
            if list(test.keys())[0] == Attribute:
                
                #A.Subset document c , retaining on the text from the row the match was found on
                #Note: this is the full string , not the information to the right or below the header
                string = c[int(j[0][0])]
                
                #B.Identify the first character of first word to right of header
                    #Syntax breakdown of StartWrd2Right
                        #Result1 = string.find(j[2][0]) # find starting position of the pattern the algo returned (the attribute header 
                        #Result2 = string[Result1+len(j[2][0]):len(string)] # return the portion of the string to the right of the header found
                        #Result3 = str(Result2.strip().split()[0]) #the first word to right of header  
                        #StartWrd2Right = string.find(Result3) #find the starting position of the word to the right of header on the line the header was found 
                        #print(StartWrd2Right)
                StartWrd2Right = string.find(str(string[string.find(j[2][0])+len(j[2][0]):len(string)].strip().split()[0])) #stripped version used to ensure white spaces are not considered in validation checks (use for control / validation tests only)
             
                #C. Find All linse below and above the current line that have a word starting in same position 
                InScopeLines = []
                for i in range(-5,5):
                    
                    #string on ith line below or above line hit was found
                    stringy = c[int(j[0][0])+i]
                   
                    #starting position of the word on the next line at position right of header  
                    StartWrd2Righty = stringy.find(str(stringy[string.find(j[2][0])+len(j[2][0]):len(string)].strip().split()[0])) #stripped version used to ensure white spaces are not considered in validation checks (use for control / validation tests only)
                    #print(StartWrd2Righty)
                    #Text on below /above lines starting from the position of the first word to right of the header
                    #  NOTE: Start/Stop characters are determined by the position of the first word to the right of the 
                    #        attribbute haeder" from the orignial "string" (returns a range of characters to use in subsetting the full line)
                    #        the string of data that is subset using this range of characters is the string i rows above/below the row the header 
                    #        was found
                    NextLineToRight= stringy[string.find(j[2][0])+len(j[2][0]):len(string)]
                  
                    #if the line below or above has the same length then append 
                    if StartWrd2Right == StartWrd2Righty:
                        InScopeLines.append(stringy[StartWrd2Right:len(stringy)].strip().split('    ')[0])
                    
                    
                    #if StartWrd2Right != string.find(str(string[string.find(j[2][0])+len(j[2][0]):len(string)].strip().split()[0])):
                    #    print('DIFF LENGTH:',string[StartWrd2Right:len(string)].strip().split('    ')[0])
                        #string.find(str(string[string.find(j[2][0])+len(j[2][0]):len(string)].strip().split()[0]))) #stripped version used to ensure white spaces are not considered in validation checks (use for control / validation tests only))
                                        
                #C.Identify location on line of the first character of that word 
                #print(string.find(str(FirstWrd2Right)))
                      
                      
               #       string.find(str(ToRight_Strip))+len(word))
                #print(InScopeLines)
            
                #all info to right delimit by multiple spaces to pull each string of text back as 1 delimited value 
                
print(InScopeLines)          

126
KIRK
130
KIRK
['FERMILAB', 'RECEIVING WAREHOUSE 2', 'KIRK RD & WILSON STREET', 'BATAVIA,IL 60510', 'United States']


In [18]:
##########################
###PARSING FOR SHIP TO####
#ADDING CONTROLS TO ABOVE#
##########################

import operator
if NoMatchFound ==0: 
    matchdetails = []
    #look through each match stored in "test" dictonary(from prior step) and determine 
    #if the expected attribute value is to the right or below the attribute header/name
    for i in test.keys():
        for j in test[i]:

            #1.Aggregated details on matched strings  
            matchdetails.append(
                                   ( int(j[0][0])                             #Line in text inwhich the match was found 
                                   , wordCount(stripwhite(c[int(j[0][0])]))   #Totalwords in line string was found
                                   , len(c[int(j[0][0])])                     #Length of characters in the line the match was found 
                                   , j[1][0]                                  #Start and end position of the matched pattern found 
                                   , j[2][0]                                  #Actual value the pattern matched on 
                                   ))

            #2.processing each match to find true result if the attribute being processed "ATTRIBUTE" is the attribute match results have been populated for(This should always be true)
            if list(test.keys())[0] == Attribute:
                
                #A.Subset document c , retaining on the text from the row the match was found on
                string = c[int(j[0][0])]
                
                #B.Identify the first character of first word to right of header
                    #Syntax breakdown of StartWrd2Right
                        #Result1 = string.find(j[2][0]) # find starting position of the pattern the algo returned (the attribute header 
                        #Result2 = string[Result1+len(j[2][0]):len(string)] # return the portion of the string to the right of the header found
                        #Result3 = str(Result2.strip().split()[0]) #the first word to right of header  
                        #StartWrd2Right = string.find(Result3) #find the starting position of the word to the right of header on the line the header was found 
                        #print(StartWrd2Right)
                StartWrd2Right = string.find(str(string[string.find(j[2][0])+len(j[2][0]):len(string)].strip().split()[0])) #stripped version used to ensure white spaces are not considered in validation checks (use for control / validation tests only)
                
            #C.Identify Controls 
                
                #Total Spaces to Right of header before first word 
                #goal: prevent pulling in a sentence of text (that has SHIP in the middle)
                Space2Right = StartWrd2Right - j[1][0][1]
     
                #Total Words to right of header
                Words2Right =  wordCount(stripwhite(string[string.find(j[2][0])+len(j[2][0]):len(string)]))
                
                #Total Groups of words to right of  header 
                Groups2Right =  wordCountDelim(stripwhite(string[string.find(j[2][0])+len(j[2][0]):len(string)]),delim="   ")
                
                print("Spaces to Right of Header: "+ str(Space2Right),'\n'
                      ,"Words to Right of Header: " + str(Words2Right),"\n"
                      ,"Groups of words seperated by 3 or more spaces :" + str(Groups2Right))
                
                #C. Find All linse below and above the current line that have a word starting in same position 
                InScopeLines = []
                for i in range(-5,6):
                    
                    #string on ith line below or above line hit was found
                    stringy = c[int(j[0][0])+i]
                   
                    #starting position of the word on the next line at position right of header  
                    StartWrd2Righty = stringy.find(str(stringy[string.find(j[2][0])+len(j[2][0]):len(string)].strip().split()[0])) #stripped version used to ensure white spaces are not considered in validation checks (use for control / validation tests only)

                    #Text on below /above lines starting from the position of the first word to right of the header
                    #  NOTE: Start/Stop characters are determined by the position of the first word to the right of the 
                    #        attribbute haeder" from the orignial "string" (returns a range of characters to use in subsetting the full line)
                    #        the string of data that is subset using this range of characters is the string i rows above/below the row the header 
                    #        was found
                    NextLineToRight= stringy[string.find(j[2][0])+len(j[2][0]):len(string)]
                  
                    #if the line below or above has the same length then append 
                    if StartWrd2Right == StartWrd2Righty:
                        InScopeLines.append(stringy[StartWrd2Right:len(stringy)].strip().split('    ')[0])

                #if you dont pull back at least 3 lines with consequtive start positions look below header 
#                if InScopeLines ==[] |  len(InScopeLines)<3: 
#                    for i in range(1:6):
#                        #string on ith line below or above line hit was found
#                        stringy = c[int(j[0][0])+i]
#                   
#                        for j in range(1:20):
                   
                    

#NOTE: loop through each line below for  each word in each row
#, and see where we see the same start position on multiple rows
#, indicating the words on these rows are aligned (grouped)

                    
                    #if StartWrd2Right != string.find(str(string[string.find(j[2][0])+len(j[2][0]):len(string)].strip().split()[0])):
                    #    print('DIFF LENGTH:',string[StartWrd2Right:len(string)].strip().split('    ')[0])
                        #string.find(str(string[string.find(j[2][0])+len(j[2][0]):len(string)].strip().split()[0]))) #stripped version used to ensure white spaces are not considered in validation checks (use for control / validation tests only))
                                        
                #C.Identify location on line of the first character of that word 
                #print(string.find(str(FirstWrd2Right)))
                      
                      
               #       string.find(str(ToRight_Strip))+len(word))
                #print(InScopeLines)
            
                #all info to right delimit by multiple spaces to pull each string of text back as 1 delimited value 
                
#print(InScopeLines)         
#print(Words2Right, Groups2Right,Space2Right)

Spaces to Right of Header: 6 
 Words to Right of Header: 5 
 Groups of words seperated by 3 or more spaces :1
Spaces to Right of Header: 5 
 Words to Right of Header: 5 
 Groups of words seperated by 3 or more spaces :1


# Process All Matches and Return Predicted Value
##### Using the information about the location of the matches , we will now loop through each match found and process it through a series of control / validation checks to ensure weed out false positives and identify the correct value for the attribute. 

## SYNTAX USED FOR NAVIGATION OF DOCUMENT 
### c: list containing all text for each row in the original pdf
        - c[0] = first line of document 
        - c[j[0][0]] = line of document match was found on 
        - c[j[0][0]+1] = line below line in which the match was found on 
        
### j: contains locational information for the match being analyzed     
        - j[0][0]  = Line in document c in which the match was found (c[j[0][0]] returns all text in row  match was found on)
        - j[1][0] = start and end position of the characters the reg-ex pattern found a match on 
        - j[2][0] = the actual string of words from line c[j[0][0]] the reg-ex pattern matched on 




## 7.Parse Attribute(PO#) from text based on  locational relationship with the attribute name and apply control check to ensure text is valid entry for attribute being extracted 

In [ ]:
#Create empty string to hold result
#First: We try to populate EndRes(the predicted value) using characters from line j[0][0] that are to the right of the attribute header the regex match was found at 
        # ToRight_Orig = string[string.find(j[2][0])+len(j[2][0]):len(string)] ==> Explanation ==> starting position of match + num characters associated with attribute being searched for : eol
#Second:If no relevant text is found to the right of the attribute header the regex matched on, we look to the lines below the header until we find the value we are intersted in 
EndRes = ""

#DEFINE MINIMUM THRESHOLDS FOR DETERMINING IF STRING TO RIGHT OF REG-EX MATCH IS VALID
#Note: 5 is used specifically for PO # and will not be valid for other attributes. 
       #these therasholds must be set based on the 'expected minimum valid' length of attribute expected. 
Words2Right = 5       #Min length of entire string to right of matched result
MinWordCharacters = 5 #Min length of the first word in string to right of matched result 

#DEFINE TOTAL NUMBER OF EXPECTED WORDS TO THE RIGHT OF THE MATCHED HEADER WE EXPECT THE ATTRIBUTE RESULTS TO HAVE
#NOTE: For PO # we only expect 1 word , for atributes like ADDRESS, this would need to be increased to refelct n multiple words
ExpectedWrdsToRight = 0 #indicates how many words to the right of the attribute header should be considered in the final value (0 = 1)

#DEFINE the maximum difference in the length of the line where match was found and the lines below this line 
#Note: This is only used in situations where the script cannot identify appropriate PO values to the right of the matched text 
percentdiflength =.05

#DEFINE VALIDATION TEST
ValidationTest = re.compile(r'(.*[a-zA-Z])?(.*[0-9])(.*[!-@#$%&*])?')

#DEFINE EXPANSION FACTOR
#this is the fraction of the total length of the below the line a match was found on that the script will use 
#to increase the total range of characters considered when looking for attributes below a specific attribute header
ExpansionFactor = 20   

#NOTE: This is a reg-ex pattern that creates a test to ensure the format of the 
#      response is in line with the format of a general response for the attribute in question
#Create Empty list to hold Desriptive sttatistics about line in which matches were found 
#Note: This is just for reference, and the resulting list is not used in any post processing 
matchdetails = []

#look through each match stored in "test" dictonary(from prior step) and determine 
#if the expected attribute value is to the right or below the attribute header/name
for i in test.keys():
    for j in test[i]:
        
        #1.Aggregated details on matched strings  
        matchdetails.append(
                               ( int(j[0][0])                             #Line in text inwhich the match was found 
                               , wordCount(stripwhite(c[int(j[0][0])]))   #Totalwords in line string was found
                               , len(c[int(j[0][0])])                     #Length of characters in the line the match was found 
                               , j[1][0]                                  #Start and end position of the matched pattern found 
                               , j[2][0]                                  #Actual value the pattern matched on 
                               ))

        #2.processing each match to find true result if the attribute being processed "ATTRIBUTE" is the attribute match results have been populated for(This should always be true)
        if list(test.keys())[0] == Attribute:
            
            #A.Subset document c , retaining on the text from the row the match was found on
            string = c[int(j[0][0])]
            print(string)
            
            #B.Identify information to the right of Attribute Name that match was found for 
            ToRight_Orig = string[string.find(j[2][0])+len(j[2][0]):len(string)] #starting position of match + num characters associated with attribute being searched for : eol
            ToRight_Strip = string[string.find(j[2][0])+len(j[2][0]):len(string)].strip() #stripped version used to ensure white spaces are not considered in validation checks (use for control / validation tests only)
            print(ToRight_Orig)
            
            #C.First Check if information to the right of Attribute Name is present / valid for a PO #
               #Note: The below checks ensures no null values are passed or sequences of words are passed to the PO Number Match Logic 
                 #1.length of entire string to right of matched result > 5 characters in length
                 #2.length of the first word in string to right of matched result > 5 characters in length  (this will be the value we pull for PO)
                 #3.String does not contain more than 2 consequtive words at bengingin of text to right of attribute found
            if len(ToRight_Strip)> Words2Right \
            and len(ToRight_Strip.split()[0])> MinWordCharacters \
            and re.search(LookupTable['MultipleWordTest'],ToRight_Strip) is None:  
                
                #IF ABOVE CONDITIONS MET, TAKE FIRST FIRST FOUND TO RIGHT OF ATTRIBUTE HEADER
                #Note: we are taking the first word because PO# is not a multi-word attribute
                       #for attributes with many words this should be changed to refelct the expected number of words. 
                if(ExpectedWrdsToRight >0):
                    nextword = ToRight_Strip.split()[:ExpectedWrdsToRight]
                else:
                    nextword = ToRight_Strip.split()[0]
                print(nextword)
            #D.RESULTS VALIDATION: for each match apply the following regex check: 
                #1.find an optional letter with anything before it
                #2.a number with anything before it
                #3.and an optional special character with anything before it
                for match in finditer(ValidationTest, nextword):
                    print(match.group())
                    EndRes = match.group() #if result is found at match i , this is populated and prevents further matches from being considered
                    print(EndRes)
        
    #3.If you cannot find any matches when looking to the right , look down at the lines below the attribute header
    if EndRes =="": 
        
        #initial value for number of lines to move down from original Attribute found(DO NOT CHANGE)
        itr=1 
        
        #loop through each match again
        for j in test[i]:
          
            #continue to move down the lines 1 at a time until the length of the orig + itr line is similar to the length 
            #of the line the attribute was found at. Becuase our text layout is preserved, the line with informaiton of PO should 
            #be of simlar length to the line the attribute header was pulled from 
            #NOTE: When comparing line lengths, we do not use the striped line, we want to preserve all leading and trailing spaces
            if len(c[int(j[0][0])+1]) < round(len(c[int(j[0][0])])-(len(c[int(j[0][0])])*percentdiflength),0):
                
                #continue to loop until the length of line j[0][0]+itr is NOT less than .95 the length of the line the initial match was found on 
                #c[int(j[0][0])+itr]: the itr'th line below the line the match was found on 
                while len(c[int(j[0][0])+itr]) < round(len(c[int(j[0][0])])-(len(c[int(j[0][0])])*percentdiflength),0):
                    
                    #if line j[0][0] + 1 does not meet above condition, we dont need to evalute it, so we place iterator at the front of the loop 
                    itr = itr + 1
                    
                    #text in itr'th line below the line the initial match was found on 
                    nextstring = c[int(j[0][0])+itr]
                    print('A',nextstring)
                    
                #else:#if the next line is of similar length to the line above, u dont need to use while loop, just take row i+1
                #    nextstring = c[int(j[0][0])+itr]
                    #print('B',nextstring)
            else: 
                #else:#if the next line is of similar length to the line above, u dont need to use while loop, just take row i+1
                nextstring = c[int(j[0][0])+itr]
                
            #remove white space from string satisfying all above conditions    
            nextstring_strip = nextstring.strip()
            print(nextstring)
            
            #length of next string
            Length = len(nextstring)
            
            #inialize itrerator to be used in expanding the range of linespace we pull back. this is done 
            #to ensure that we pull back one complete word 
            #NOTE: if you need to pull back multiple words then u need to change the ExpectedWrdsToRight variable 
            itr=2 #initialize i
             
            #if there are 0 words picked up in the text below(if its just blank space below)
            if wordCount(stripwhite(nextstring[int(j[1][0][0])-itr:j[1][0][1]+itr])) < ExpectedWrdsToRight+1:
                
                #Continue to expand the space on the line considered until there is at least 'ExpectedWrdsToRight' word(s)
                #Note: We expand the range of characters considered UNIFORMLY on each side of the initital range)
                while wordCount(stripwhite(nextstring[int(j[1][0][0])-itr:j[1][0][1]+itr])) < ExpectedWrdsToRight+1:
                    
                    #if first iteration just expand the range slightly to see if we are able to find result
                    if itr==2: 
                        
                        #pull in positioin of header +/- 2 char for first try
                        Below_Text = nextstring[int(j[1][0][0])-itr:j[1][0][1]+itr] 
                        
                        #reset itr for start of all subsequent loops to be 1/20'th the total length of the line being considered
                        itr= int(round(Length / ExpansionFactor,0)) 
            
                    #expand the region considered out by a factor of 1/20th the total characters in the line until at least 2 words are present 
                    Below_Text = nextstring[int(j[1][0][0])-itr:j[1][0][1]+itr]
                    itr= itr + int(round(Length/ExpansionFactor,0))
            else: 
                 for i in range(0,20,2):
                        if wordCount(stripwhite(nextstring[int(j[1][0][0])-i:j[1][0][1]+i])) <= 1:
                            Below_Text = nextstring[int(j[1][0][0])-i:j[1][0][1]+i] #pull in positioin of header +/- 2 char for first try
                            print(Below_Text)
                        
                
            #Loop through all matches and extract the associated PO Number 
            for match in finditer(ValidationTest, Below_Text):
                    print(match.group())
                    EndRes = stripwhite(match.group())

#print the end result
EndRes



## 8.Company Specific Table Parsing 
### Script Built For: Grey Matters Inc.

##### Note: I duplicate some key queries from above, once script is enclosed in function we will need to have it iterate through each variable using the same script, its a waste to repeat yourself! ####

### 1.Find Start Location / Row of Table 
##### Use regex rules searching for presence of column names associated with the table

In [19]:
#NOTE: TEST contains all matches for the element in question
#identify start and end locaiton of string found for each line in po 
test ={}
list1 = []

#for every line in the PO 
    # apply regex pattern associated with attribute in question 
    # and for each pattern match found in line i generate locational information 
    # to use to parse / find attribute value 
for i in range(len(c)):
       
    #check for match in each line of the rule associated with "TABLE" key
    for match in finditer(LookupTable['TABLE'], c[i]):
        
        #output nested list with tuple of locational information about the match 
        #create a 3 element tuple(line #, where match was found, what match was) with the information on locaiton of string
        list1.append(([str(i)],[match.span()], [match.group()]))

#assign dictonary values for "TABLE" KEY from results of lookuptable dic stored in the list above to a new dict with same Key ('TABLE') holding results of looping through each line in doc and applying rule from fist dictonary 
#
#0 = PO , 1=Mulitwordtest ,2 = Table, 3 = tbleend1 , 4 = tableend2
#

#we refecrence list(LookupTable.keys())[2] simply to ensure we use the same naming convention in the new list
test[list(LookupTable.keys())[2]] = list1

print(test)

{'COMPANY': []}


### 2.Identify Start / End location of each header in the table

##### Note: Table Column widths are consistent, so we can pre-define the column width associated with each header to improve accuracy(at the cost of flexibiilty) if needed 

In [ ]:
import operator
for i in test.keys():
    for j in test[i]:
        
        #header line (actual string of text where match was found)
        line = c[int(j[0][0])]
        
        #split line into groups of words (seperated by like 4 or 5 spaces) 
        split = line.split('   ')
        
        #remove blank elemnts from list
        split = list(filter(None, split))
        
        #loop through each header value and identify the start and end location of the string 
        StartLoc = []
        for z in range(len(split)):
            
            #header word
            word = split[z]
            
            #start and end loc of each word in the line being parsed
            StartLoc.append([stripwhite(str(word)), int(j[0][0]) ,line.find(str(word)), line.find(str(word))+len(word)])    
print(StartLoc)                      

### 3.Identify the ending location of the table 
##### Note: Greymatter tables always end after Total or Subtotal line . First we check subtotal because that will always appear before total, if subtotal is not found we revert to finding "Total"

In [ ]:
#Find Ending location of Table (GreyMatter: either line above subtotal or line above total)
list2 = []
test2 = {}

#Indicators 
Grey1 = None
Grey2 = None 

for i in range(len(c)):
    
    #First look for SUBTOTAL becuase it will always be first(if its there)
    for match in finditer(LookupTable['GREYTABLEEND1'], c[i]): 
        if match is None: #exit loop if no match 
            break    
            
        #output nested list with tuple of locational information about the match 
        list2.append(([str(i)],[match.span()], [match.group()]))#create a 3 element tuple with the information on locaiton of string
        
        #indicator (indicates subtotal found)
        Grey1 = 1
        
for i in range(len(c)): 
    
    #is None: #If you dont find subtotal look for TOTAL 
    if list2 ==[]:
        for match in finditer(LookupTable['GREYTABLEEND2'], c[i]):
            #output nested list with tuple of locational information about the match 
            list2.append(([str(i)],[match.span()], [match.group()]))#create a 3 element tuple with the information on locaiton of string
            #indicator 
            Grey2 = 1
        
        
#assign obect to dictonary? 
if Grey1==1:
    #test2[list(LookupTable.keys())[3]] = list2
    test2['GREYTABLEEND1'] = list2
    
    endvar = stripwhite(test2['GREYTABLEEND1'][0][2][0])
    print(endvar)
    print("SubTotal Found")
    
if Grey2==1: 
    test2['GREYTABLEEND2'] = list2
    endvar = stripwhite(test2['GREYTABLEEND2'][0][2][0])
    print(endvar)
    print("Total Found")
    
#print matchesi
for i in test2.keys():
    for j in test2[i]:
        print(j)
        
#Begining / Ending Line of table 
TableEnd = int(list2[0][0][0])
TableBeg = int(list1[0][0][0])
TableRange = TableEnd - TableBeg

#total rows in table 
TableRange
print(TableBeg)
print(TableEnd)

In [ ]:
list(LookupTable.keys())[0]

### 4.Extract Table information using pandas 

In [ ]:
from prettytable import PrettyTable
import pandas as pd
#empty list for results
TableInfo = []
pdframe = None

#loop through each line of the table and extract information 
for line in range(TableBeg,TableEnd):
        Thisline = c[line]
        Nextline = c[line+1]
        
        print(Thisline)
        #print(Nextline)
        #split line into words 
        split = Thisline.split('  ')
        #print(split)
        
        #remove blank elemnts from list
        split = list(filter(None, split))
        #strip white space from each element in list
        #split = [x.strip(' ') for x in split]
        
        #print(split[0])
        #if its the header line of the table split all groups (no ordering needed)
        if line ==TableBeg: 
            t = PrettyTable(split)
            TotalCols = len(split)
            TableVals = (["MISSING"],)*TotalCols
            
            #save as dataframe to allow easy manipulation of output
            pdframe = pd.DataFrame(columns = [split])
         
        if line > TableBeg and len(split)>1:
     
            #itr = len(split)
            #IF FIRST ELEMENT IN SPLIT IS A NUMERIC THEN U KNOW U FOUND A NEW LINE ENTRY(NOTE THIS WILL ONLY CAPTURE THE FIRST LINE OF THE NEW ENTRY)
            #print(itr)
            #for each column in TableVals
            
            #if  re.match(r'^[1-9][0-9]*$', stripwhite(split[0])) is not None \
            #    or  re.match(r'^[1-9][0-9]*$', stripwhite(split[1])) is not None:
            if split[0] != '': 
            #if len(stripwhite(split[0]))<2 #make this a regex condition for any number ^[1-9][0-9]*$
            
                for group in range(len(TableVals)):
                    if group == 0 and len(split)>0:  #1th element of Tablevals (min)
                        TableVals = (stripwhite(split[0]),)+TableVals[1:]
                        split = [x for x in split if x != split[0]] #remove element from split that has been acounted for

                    if group == 1  and len(split)>0:  #6th element of table vals (max)
                        TableVals = TableVals[0:TotalCols-1]+(split[len(split)-1],)
                        split = [x for x in split if x != split[len(split)-1]]

                    if group == 2  and len(split)>0:  #2nd element of table vals (min)
                        TableVals=  TableVals[:1]+(split[0],)+TableVals[2:]
                        split = [x for x in split if x != split[0]]

                    if group == 3  and len(split)>0:  #5th elment of table vals(max)
                        TableVals = TableVals[:TotalCols-2]+(split[len(split)-1],)+TableVals[len(TableVals)-1:]
                        split = [x for x in split if x != split[len(split)-1]]

                    if group == 4  and len(split)>0:  #3th elment of table vals(min)
                        TableVals=  TableVals[:2]+(split[0],)+TableVals[3:]
                        split = [x for x in split if x != split[0]]

                    if group == 5  :  #3th elment of table vals(min)
                        if len(split)==0:
                            TableVals=TableVals[:TotalCols-3]+('MISSING DATA',)+TableVals[len(TableVals)-2:]
                        else:        
                            TableVals=  TableVals[:TotalCols-3]+(split[len(split)-1],)+TableVals[len(TableVals)-2:]
                            split = [x for x in split if x != split[0]]

                t.add_row(TableVals)
                pdframe.loc[len(pdframe)+1] = TableVals

                
#rename all headers so no white space is left 
pdframe.columns = pdframe.columns.str.strip()
print(pdframe)



### 5.Post Processing of table information(Correct errors identified in table parsing) 

In [ ]:

#post checks 
#1.split quanity and description if quanitity has characters and numeric as first character in string 
#Rule: if description is missing(could not find multiple spaces between description and some other field)
#      and the feild before description has length > 10 (no quanitity should be this much)
#      and the first charcter in quantity is a digit \

for index,row in pdframe.iterrows():
    if  row['Description'][0] == 'MISSING'  \
    and len(row['Quantity'])>10 \
    and  re.match(r'(\s)?\d',(row['Quantity'][:2])) is not None: 

        row['Description'] = row['Quantity'][2:]
        row['Quantity']= row['Quantity'][:2]

print(pdframe)

### 4.Option2 (Leverage Static position of table information in regards to column width)

##### Note: this enables retrieval of all information for each column instead of just the rows associated with the start line for a new item(the rows with the pricing infomration) 

In [ ]:
from prettytable import PrettyTable
import pandas as pd
import numpy as np

#empty list for results
TableInfo = []

pdframe = None
#loop through each line of the table and extract information 

if endvar =='Subtotal':
    
    #create empty vector for each column 
    Line = []
    Quantity=[]
    Item=[]
    Description=[]
    UnitPrice =[]
    LineTotal =[]
        
    for line in range(TableBeg,TableEnd):
        Thisline = c[line]
        Nextline = c[line+1]
       
        #print(Thisline)
            #if its the header line of the table split all groups (no ordering needed)
        if line ==TableBeg: 
                
            #split line into words 
            split = Thisline.split('  ')

            #remove blank elemnts from list
            split = list(filter(None, split))
             
            t = PrettyTable(split)
            TotalCols = len(split)
            TableVals = (["MISSING"],)*TotalCols
            
            #save as dataframe to allow easy manipulation of output
            pdframe = pd.DataFrame(columns = [split])

        if line > TableBeg and len(split)>1:
            
            Line.append((line,Thisline[:7]))
            Quantity.append((line,Thisline[8:19]))
            Item.append((line,Thisline[20:32]))
            Description.append((line,stripwhite(Thisline[33:55])))
            UnitPrice.append((line,Thisline[75:105]))
            LineTotal.append((line,Thisline[106:]))

    #rename all headers so no white space is left 
    pdframe.columns = pdframe.columns.str.strip()

    #create individual dataframes for each column(may not be needed)
    Line2 = pd.DataFrame(data = Line, columns =["Index","Line"])
    Quantity2 = pd.DataFrame(data = Quantity, columns =["Index","Quantity"])
    Item2 = pd.DataFrame(data = Item, columns =["Index","Item"])
    Description2 = pd.DataFrame(data = Description, columns =["Index","Description"])
    UnitPrice2 = pd.DataFrame(data = UnitPrice, columns =["Index","UnitPrice"])
    LineTotal2 = pd.DataFrame(data = LineTotal, columns =["Index","LineTotal"])

    #merge columns back together into final table 
    result = pd.merge(left =Line2,right = Quantity2, how="left", on="Index")
    result2 = pd.merge(left =result,right = Item2, how="left", on="Index")
    result3 = pd.merge(left =result2,right = Description2, how="left", on="Index")
    result4 = pd.merge(left =result3,right = UnitPrice2, how="left", on="Index")
    result5 = pd.merge(left =result4,right = LineTotal2, how="left", on="Index")
    print(result5)

    
if endvar =='Total':
    #create empty vector for each column 
    Item=[]
    Quantity=[]
    Description=[]
    Rate =[]
    Amount =[]
        
    for line in range(TableBeg,TableEnd):
        Thisline = c[line]
        Nextline = c[line+1]
       
        #print(Thisline)
            #if its the header line of the table split all groups (no ordering needed)
        if line ==TableBeg: 
                
            #split line into words 
            split = Thisline.split('  ')

            #remove blank elemnts from list
            split = list(filter(None, split))
             
            t = PrettyTable(split)
            TotalCols = len(split)
            TableVals = (["MISSING"],)*TotalCols
            
            #save as dataframe to allow easy manipulation of output
            pdframe = pd.DataFrame(columns = [split])

        if line > TableBeg and len(split)>1:
            
            #variable by comapny / client need
            Item.append((line,Thisline[:21]))
            Quantity.append((line,Thisline[22:36]))
            Description.append((line,stripwhite(Thisline[37:95])))
            Rate.append((line,Thisline[96:115]))
            Amount.append((line,Thisline[116:]))

    #rename all headers so no white space is left 
    pdframe.columns = pdframe.columns.str.strip()

    #create individual dataframes for each column(may not be needed)
    #Line2 = pd.DataFrame(data = Line, columns =["Index","Line"])
    Quantity2 = pd.DataFrame(data = Quantity, columns =["Index","Quantity"])
    Item2 = pd.DataFrame(data = Item, columns =["Index","Item"])
    Description2 = pd.DataFrame(data = Description, columns =["Index","Description"])
    UnitPrice2 = pd.DataFrame(data = Rate, columns =["Index","Rate"])
    LineTotal2 = pd.DataFrame(data = Amount, columns =["Index","Amount"])

    #merge columns back together into final table 
    result = pd.merge(left =Item2,right = Quantity2, how="left", on="Index")
    result2 = pd.merge(left =result,right = Description2, how="left", on="Index")
    result3 = pd.merge(left =result2,right = UnitPrice2, how="left", on="Index")
    result4 = pd.merge(left =result3,right = LineTotal2, how="left", on="Index")
    #result5 = pd.merge(left =result4,right = LineTotal2, how="left", on="Index")
    print(result4)
    

result4[:4]

## Functions 

In [1]:

##Function to determine if pdf is searchable 
def IsPDFSearchable(path): 
    
    #determine if pdf is searchable 
    import os, re
    
    #path is the directory with the files, other 2 are the names of the files you will store your lists in
    path = path
    
    #use this if u want to actually create a file holding all pdfs that are searchable vs not 
    #files_with_text = open("C://Users//caridza//Desktop//EY//AI//OCR//GE PO RFP//solution data//PO Examples//files_with_text.txt", 'a')
    #image_only_files = open("C://Users//caridza//Desktop//EY//AI//OCR//GE PO RFP//solution data//PO Examples//image_only_files.txt", 'a')
    
    files_with_text = []
    image_only_files = []
    
    #have os make a list of all files in that dir for a loop
    filelist = os.listdir(path)
    
    #compile regular expression that matches "Font"
    mysearch = re.compile(r'.*Font.*', re.DOTALL) #.decode("utf-8")
    
    #loop over all files in the directory, open them in binary ('rb'), search that binary for "Font"
    #if they have "Font" they have text, if not they don't
    #(pdf does something to understand the Font type and uses this word every time the pdf contains text)
    for pdf in filelist:
        openable_file = os.path.join(path, pdf)
        cat_file = open(openable_file, 'rb')
        usable_cat_file = cat_file.read()
        #print usable_cat_file
        if mysearch.match(str(usable_cat_file)):
            #files_with_text.write(pdf + '\n') #use if u are storing info in file
            files_with_text.append(pdf)
        else:
            #image_only_files.write(pdf + '\n') #use if u are storing info in file 
            image_only_files.append(pdf)
    
    #use if storing pdf names in files
    #close files holding searchable vs unsearchable pdfs
    #files_with_text.close()
    #image_only_files.close()
    return files_with_text, image_only_files


##https://www.freebsd.org/cgi/man.cgi?query=pdftotext&apropos=0&sektion=0&manpath=FreeBSD+6.2-RELEASE+and+Ports&format=html
#http://www.pdftron.com/assets/pdfs/support/PDFTron_PDF2Text_User_Manual.pdf
#options not used 
##'-remove_hidden_text',
##'-remove_invisible_text'
## pdf2text -c 150,600,250,700 license.pdf -a 1 #extract text from a speicific region 
def get_bbox_file(pdf_path, txt_path):
    if not os.path.exists(pdf_path):
        raise FileNotFoundError("Couldn't find {}".format(pdf_path))
    elif os.path.exists(txt_path):
        return txt_path
    # TXT file doesn't exist yet. Create it.
    try:
        p1 = Popen(('pdftotext','-bbox-layout',  
                       pdf_path, txt_path),
                      stdin=PIPE, stdout=PIPE)
        output = p1.communicate()
    finally:
        p1.stdout.close()
        p1.stdin.close()
    try:
        p1.terminate()
        p1.kill()
    except:
        pass

    return txt_path
    
#return tables 
#return bounding boxes
#for i in range(len(SearchablePDFPaths)):
#    bboxpath = croppath+str(SearchablePDFPaths[i].rsplit("//", 1)[1]).replace(".pdf","_BBox.txt")
#    pdfpath = SearchablePDFPaths[i]
#    bboxtextpaths.append(get_bbox_file(pdfpath, bboxpath))

#extract only bounding box information
#lines = []
#for i in range(len(bboxtextpaths)):
#    #lines.append([ line for line in open(bboxtextpaths[i]) if 'block' in line])
#    lines.append([ line.replace("</block>\n","").replace("<block","").replace(">\n","").replace(",","") for line in open(bboxtextpaths[i]) if 'block' in line])

def get_text_file(pdf_path, txt_path):
    if not os.path.exists(pdf_path):
        raise FileNotFoundError("Couldn't find {}".format(pdf_path))
    elif os.path.exists(txt_path):
        return txt_path
    # TXT file doesn't exist yet. Create it.
    try:
        p1 = Popen(('pdftotext','-layout',  
                       pdf_path, txt_path),
                      stdin=PIPE, st=PIPE)
        output = p1.communicate()
    finally:
        p1.stdout.close()
        p1.stdin.close()
    try:
        p1.terminate()
        p1.kill()
    except:
        pass

    return txt_path

#get files
def getfiles(path, extension):
    items = os.listdir(path)
    newlist = []
    for names in items:
        if names.endswith(extension):
            newlist.append(names)
    return newlist


def isPhoneNumber(text, chars = 11):
    if len(text) != chars:
        return False
    
    for i in range(0, 2):
        if not text[i].isdecimal():
            return False
        #if text[3] != '-':
        #    return False
    for i in range(3, 6):
        if not text[i].isdecimal():
            return False
        #if text[7] != '-':
        #    return False
    for i in range(chars,11):
        if not text[i].isdecimal():
            return False
    return text


def validate(email): 
    match=re.search(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9]+\.[a-zA-Z0-9.]*\.*[com|org|edu]{3}$)",email)
    if match:
        return 'Valid email.'
    else:
        return 'Invalid email.'
    

#count words in string 
def wordCount(mystring):  
    tempcount = 0  
    count = 1  
    indicator = 0  
    try:  
        for character in mystring:  
            if character == " ":  
                tempcount +=1  
                if tempcount ==1:  
                    count +=1  

                else:  
                    tempcount = 0  
        return count  

    except Exception:  
        error = "Not a string"  
        return error  


#count words in string 
def wordCountDelim(mystring,delim=" "):  
    tempcount = 0  
    count = 1  
    indicator = 0  
    try:  
        for character in mystring:  
            if character == delim:  
                tempcount +=1  
                if tempcount ==1:  
                    count +=1  

                else:  
                    tempcount = 0  
        return count  

    except Exception:  
        error = "Not a string"  
        return error  
    
#remove all white space from string (only use to count total words in line)
def stripwhite(string):

    string = string.rstrip().lstrip() # Remove all extra spaces at the start and at the end of the string
    while "  " in string: # While  there are 2 spaces beetwen words in our string...
        string = string.replace("  ", " ") # ... replace them by one space!
    return(string)

